在 AI 相关的面试中，经常会有面试官让写 self-attention，但是因为 transformer 这篇文章其实包含很多的细节，因此可能面试官对于 self-attention 实现到什么程度是有不同的预期。因此这里想通过写不同版本的 self-attention 实现来达到不同面试官的预期。以此告诉面试官，了解细节，但是处于时间考虑，可能只写了简化版本，如果有时间可以把完整的写出来。

# 自注意力(Self-Attention)机制理论

自注意力机制是Transformer架构的核心组成部分，它允许模型在处理序列数据时捕捉长距离依赖关系。与传统的RNN和CNN不同，自注意力机制可以直接建立序列中任意两个位置之间的联系，而不受位置距离的限制。

## 基本原理

自注意力的核心思想是：对于序列中的每个元素，通过计算它与序列中所有元素（包括自身）的关联程度，生成一个加权的表示。这种机制使模型能够"关注"序列中的相关部分，无论它们在序列中的位置如何。

## 数学定义

设输入序列表示为矩阵 $X \in \mathbb{R}^{n \times d}$，其中 $n$ 是序列长度，$d$ 是特征维度。自注意力机制的计算过程如下：

1. **线性投影**：首先，将输入 $X$ 投影到三个不同的空间，生成查询(Query)、键(Key)和值(Value)矩阵：
   $$Q = XW^Q, \quad K = XW^K, \quad V = XW^V$$
   
   其中 $W^Q, W^K, W^V \in \mathbb{R}^{d \times d_k}$ 是可学习的参数矩阵，$d_k$ 是投影后的维度。

2. **注意力分数计算**：计算查询和键之间的相似度得到注意力分数矩阵：
   $$S = QK^T = XW^Q(XW^K)^T \in \mathbb{R}^{n \times n}$$
   
   矩阵 $S$ 中的每个元素 $S_{ij}$ 表示位置 $i$ 对位置 $j$ 的注意力分数。

3. **缩放**：为了防止梯度消失问题，对注意力分数进行缩放：
   $$S_{scaled} = \frac{S}{\sqrt{d_k}} = \frac{QK^T}{\sqrt{d_k}}$$
   
   这里的 $\sqrt{d_k}$ 是缩放因子，$d_k$ 是键向量的维度。

4. **掩码(可选)**：在某些情况下（如解码器中的自回归生成），需要防止位置 $i$ 获取到位置 $j > i$ 的信息，此时可以应用掩码：
   $$S_{masked} = S_{scaled} \odot M$$
   
   其中 $M \in \mathbb{R}^{n \times n}$ 是掩码矩阵，$\odot$ 表示元素乘法。

5. **Softmax归一化**：对缩放后的注意力分数应用softmax函数，将其转换为概率分布：
   $$A = \text{softmax}(S_{scaled}) = \frac{\exp(S_{scaled})}{\sum \exp(S_{scaled})}$$
   
   其中，softmax按行应用，确保每行的和为1。

6. **加权求和**：使用注意力权重对值矩阵进行加权求和，得到最终的输出：
   $$O = AV = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

综合起来，自注意力机制可以表示为以下公式：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

## 多头注意力(Multi-Head Attention)

为了增强模型的表示能力，Transformer使用了多头注意力机制，即同时学习多组不同的线性投影：

1. 对于第 $i$ 个头，计算：
   $$Q_i = XW_i^Q, \quad K_i = XW_i^K, \quad V_i = XW_i^V$$
   $$\text{head}_i = \text{Attention}(Q_i, K_i, V_i)$$

2. 将所有头的输出拼接并通过一个线性变换：
   $$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O$$

其中 $h$ 是头的数量，$W^O \in \mathbb{R}^{hd_k \times d}$ 是输出投影矩阵。

多头注意力允许模型同时关注来自不同表示子空间的信息，丰富了表示能力。

## 计算复杂度

自注意力机制的时间复杂度为 $O(n^2 \cdot d)$，其中 $n$ 是序列长度，$d$ 是特征维度。空间复杂度同样为 $O(n^2 \cdot d)$。这意味着对于非常长的序列，计算成本会变得很高，这也是后续各种改进版本自注意力机制的主要优化方向。

下面代码是参考b站视频的从零开始编写注意力机制的代码：
b站视频：https://www.bilibili.com/video/BV19YbFeHETz/?spm_id_from=333.1387.homepage.video_card.click&vd_source=01e76602bca90a928935ecc928b2c476

### 写法1：简单写法

In [8]:
import math
import torch
import torch.nn as nn

class SelfAttention1(nn.Module):
    def __init__(self, hidden_dim:int = 728):
        super().__init__()
        self.hidden_dim=hidden_dim
        #初始化q,k,v的投影矩阵
        self.query_proj=nn.Linear(hidden_dim,hidden_dim)
        self.key_proj=nn.Linear(hidden_dim,hidden_dim)
        self.value_proj=nn.Linear(hidden_dim,hidden_dim)


    def forward(self,x):
        #x: (batch_size, seq_len, hidden_dim)
        Q=self.query_proj(x)
        K=self.key_proj(x)
        V=self.value_proj(x)
        #attenion_value: (batch_size, seq_len, seq_len)
        attention_value=torch.matmul(Q,K.transpose(-1,-2) )
        print("注意力分数：",attention_value)
        attention_weight=torch.softmax(attention_value/math.sqrt(self.hidden_dim),dim=-1)
        #output: (batch_size, seq_len, hidden_dim)
        output=torch.matmul(attention_weight,V)
        return output

        
X=torch.rand(3,2,4)
print(X)
net=SelfAttention1(4) #输入参数为init时候的hidden_dim
net(X)
        



tensor([[[0.0629, 0.5470, 0.6495, 0.3939],
         [0.2147, 0.9538, 0.2387, 0.8803]],

        [[0.4591, 0.1322, 0.7582, 0.1914],
         [0.1903, 0.4623, 0.6657, 0.3359]],

        [[0.7230, 0.8849, 0.8254, 0.7832],
         [0.5898, 0.4862, 0.6398, 0.1634]]])
注意力分数： tensor([[[-0.1867, -0.2050],
         [-0.2925, -0.3349]],

        [[-0.0601, -0.0154],
         [-0.1062, -0.1205]],

        [[-0.0906, -0.0132],
         [-0.0093,  0.0235]]], grad_fn=<UnsafeViewBackward0>)


tensor([[[-0.9080,  0.1165,  0.0589, -0.3707],
         [-0.9070,  0.1157,  0.0595, -0.3704]],

        [[-0.7746, -0.1408,  0.1270, -0.2042],
         [-0.7747, -0.1419,  0.1266, -0.2033]],

        [[-1.0428, -0.0289,  0.0970, -0.1738],
         [-1.0449, -0.0283,  0.0962, -0.1743]]], grad_fn=<UnsafeViewBackward0>)

### 代码实现过程中的补充知识点：
#### 1.三维张量的矩阵乘法规则

在深度学习框架（如PyTorch、TensorFlow）中，当对形状为 (B, M, N) 和 (B, N, P) 的两个三维张量进行矩阵乘法时：第一个维度（批次维度B）被视为独立的批次。对每个批次，执行普通的矩阵乘法（按照二维矩阵乘法规则），结果形状为 (B, M, P)。

能够进行矩阵乘法的关键因素是遵循矩阵乘法的维度对齐规则，具体来说，对于两个矩阵 A 和 B 的乘法 A @ B：
- A 的列数必须等于 B 的行数
- 结果矩阵的形状将是 (A的行数, B的列数)

对于批量矩阵乘法（如三维张量 A 和 B 的乘法）：
1. **批次维度必须匹配或可广播**：两个张量的第一个维度（批次维度）必须相同，或者其中一个是1（可以广播）
2. **内部维度必须匹配**：A 的最后一个维度必须等于 B 的倒数第二个维度
3. **结果形状**：(批次大小, A的倒数第二个维度, B的最后一个维度)



在你的例子中：

第一次乘法：
- Q: (batch_size, seq_len, hidden_dim)
- K.transpose: (batch_size, hidden_dim, seq_len)
- 对齐点：Q 的最后一个维度 (hidden_dim) = K.transpose 的倒数第二个维度 (hidden_dim)
- 结果：(batch_size, seq_len, seq_len)

第二次乘法：
- attention_weight: (batch_size, seq_len, seq_len)
- V: (batch_size, seq_len, hidden_dim)
- 对齐点：attention_weight 的最后一个维度 (seq_len) = V 的倒数第二个维度 (seq_len)
- 结果：(batch_size, seq_len, hidden_dim)



### 写法2 合并计算



In [9]:
import torch
import torch.nn as nn
import math

class SelfAttention2(nn.Module):
    def __init__(self, dim:int=728):
        super().__init__()
        self.dim=dim
        #KV 矩阵计算的时候，可以合并成一个大矩阵计算。
        self.qkv=nn.Linear(dim, dim*3)
        self.output=nn.Linear(dim, dim)

    def forward(self,x):
        QKV=self.qkv(x)
        #把QKV拆分成Q,K,V
        Q,K,V=torch.split(QKV,self.dim,dim=-1)
        #dim=-1表示在最后一个维度上做softmax，也就是在最后一个维度上做归一化
        attention_weight=torch.softmax(torch.matmul(Q,K.transpose(-1,-2)/math.sqrt(self.dim)),dim=-1)
        prin
        output=attention_weight@V
        return output
        

    
X=torch.rand(3,2,4)
print(X)
net=SelfAttention2(4) #输入参数为init时候的hidden_dim
net(X)
        
        

tensor([[[0.2709, 0.3968, 0.7889, 0.8579],
         [0.8249, 0.4581, 0.3383, 0.5962]],

        [[0.2547, 0.4830, 0.6664, 0.0682],
         [0.5068, 0.5681, 0.5868, 0.4327]],

        [[0.0549, 0.2919, 0.9394, 0.2116],
         [0.3851, 0.8969, 0.9019, 0.8795]]])


NameError: name 'prin' is not defined

### 写法3：加入dropout和attetion mask

In [ ]:
import torch
import torch.nn as nn
import math


class SelfAttention3(nn.Module):
    def __init__(self, dim,dropout=0.1,*args,**kwargs)->None:
        super().__init__(*args,**kwargs)
        self.dim=dim
        self.attention_dropout=nn.Droupout(droupout)
        self.output=nn.Linear(dim,dim)


### 一些对attetion的深入理解


注意力分数是自注意力机制中衡量序列中不同位置之间关联强度的数值。它是通过查询向量(Query)和键向量(Key)的点积计算得到的中间结果。

#### 数学表示

在自注意力计算流程中，注意力分数的计算公式为：

$$S = QK^T = XW^Q(XW^K)^T$$

其中，$S$是一个$n \times n$的矩阵，$n$是序列长度。矩阵$S$中的每个元素$S_{ij}$代表位置$i$对位置$j$的注意力分数。

#### 注意力分数的含义

注意力分数从本质上代表了**"当前位置应该关注序列中哪些其他位置"**的重要性权重。具体来说：

1. **高分数**：表示两个位置之间有很强的语义关联，当前位置应该高度关注该位置的信息
2. **低分数**：表示两个位置关联度低，当前位置可以忽略该位置的信息

#### 直观理解

可以从不同角度理解注意力分数：

1. **信息检索视角**：
   - Query是"搜索查询"
   - Key是"文档关键字"
   - 注意力分数反映了查询与各个关键字的匹配程度

2. **语言理解视角**：
   - 在自然语言中，注意力分数可以捕捉：
     - 代词与其指代对象的关系
     - 依存关系
     - 语义上相关的词汇

3. **特征集成视角**：
   - 每个位置都在"询问"：序列中哪些位置包含与我相关的信息？
   - 注意力分数给出了回答：这些位置的信息与你相关的程度

#### 注意力分数的使用流程

1. **计算原始分数**：$S = QK^T$
2. **缩放**：$S_{scaled} = \frac{S}{\sqrt{d_k}}$（防止维度较高时梯度消失）
3. **应用掩码**(如需)：屏蔽不应关注的位置（如填充位置或未来信息）
4. **Softmax归一化**：$A = \text{softmax}(S_{scaled})$，转换为概率分布
5. **加权汇总**：用这些概率对值向量进行加权求和 $O = AV$

#### 示例

假设在一个句子"猫坐在垫子上，它很舒适"中：

- 分析"它"这个位置时，注意力分数会对"猫"给予很高的权重，表明这两个位置高度相关
- 而对"垫子"、"上"等词给予较低的权重
- 这个机制使模型能够"理解"代词"它"指代的是"猫"

通过注意力分数机制，Transformer系列模型能够有效捕捉序列数据中的长距离依赖关系，这是它们成功的关键因素之一。